In [1]:
from xml.dom.minidom import parseString
from bs4 import BeautifulSoup
import pandas as pd
import dicttoxml
import requests
import re

In [2]:
url = 'https://habr.com/ru/'

r=requests.get(url).text
soup=BeautifulSoup(r,'lxml')

l=soup.find_all('li',class_='content-list__item content-list__item_post shortcuts_item')

In [4]:
print(len(l))
for i,j in enumerate(l):
    try:
        j.find('span',class_='user-info__nickname user-info__nickname_small').text
    except:
        del l[i]
print(len(l))

18
18


In [5]:
def make_rubric(li):
    di = dict()
    for i,j in enumerate(li.find_all('li',class_='inline-list__item')):
        try:
            di['rubric id="'+str(i)+'"']={'title':j.text.replace(',','').replace('\n',''),
                                     'link':j.find('a').get('href')}
        except:
            di['rubric id="'+str(i)+'"']={'title':j.text.replace(',','').replace('\n',''),
                                     'link':'None'}
    return di

def make_voiting(li):
    di=dict()
    di['total']=re.search(r'\d+',li.find('span',class_='post-stats__result-counter').get('title')).group()
    di['up']=re.search(r'↑\d+',li.find('span',class_='post-stats__result-counter').get('title')).group().replace('↑','')
    di['down']=re.search(r'↓\d+',li.find('span',class_='post-stats__result-counter').get('title')).group().replace('↓','')
    di['result']=li.find('span',class_='post-stats__result-counter').text
    return di

In [6]:
d=dict()
for i in l:
    print(i.find('span',class_='user-info__nickname user-info__nickname_small').text)
    d['post '+'id="'+i.get('id')+'"']={
                'name':i.find('span',class_='user-info__nickname user-info__nickname_small').text,
                'post_time':i.find('span',class_='post__time').text,
                'post_title':i.find('a',class_='post__title_link').text,
                'inline_list':make_rubric(i),
                'sinopsis':i.find('div',class_='post__text').text.replace('\xa0',' '),
                'voting':make_voiting(i)}

ru_vds
kern-panic
the_bat
Bee_brightside
Dmytro_Kikot
inetstar
gavrashenko
kventinel
nckma
e_finkel
adamant
ArtLop
Galilea
PatientZero
Sound_cULT
sanders1967
smartdev
irishaspir


In [14]:
d['post id="post_493712"']

{'name': 'ru_vds',
 'post_time': 'вчера в 12:30',
 'post_title': 'Новые возможности TypeScript, повышающие удобство разработки',
 'inline_list': {'rubric id="0"': {'title': 'Блог компании RUVDS.com         ',
   'link': 'https://habr.com/ru/company/ruvds/'},
  'rubric id="1"': {'title': 'Разработка веб-сайтов         ',
   'link': 'https://habr.com/ru/hub/webdev/'},
  'rubric id="2"': {'title': 'TypeScript',
   'link': 'https://habr.com/ru/hub/typescript/'},
  'rubric id="3"': {'title': 'Перевод', 'link': 'None'}},
 'sinopsis': 'TypeScript, во многих отношениях, больше похож не на язык программирования, а на мощный инструмент для линтинга и документирования кода, который помогает писать более качественные JavaScript-программы.\n\r\nОдна из наиболее заметных сильных сторон TypeScript — это поддержка некоторых из новейших возможностей, описанных в спецификации ECMAScript. Когда разработчик обновляется до новой версии TypeScript, это означает, что в его распоряжении оказываются и новые во

In [7]:
pd.json_normalize(d)
l=list()
for i in list(d.values()):
    l.append(pd.json_normalize(i).values[0])
    
s=''
for i in l:
    s+=' '.join(i)
    s+='\n--------------------------\n'
    
f = open('dict_to_text.txt','w',encoding='utf-8')
try:
    f.write(s)
finally:
    f.close()

In [8]:
xml = dicttoxml.dicttoxml(d,attr_type=False)
doc = parseString(xml)
doc.toprettyxml()
f = open("output.xml", "w",encoding='utf-8')
try:
    f.write(doc.toprettyxml(indent="  ").replace('<key name="post id=&quot;','<post id="').replace('<key name="rubric id=&quot;','<rubric id="').replace('&quot;','').replace('</link>\n','</link>\n      </rubric>\n').replace('\n      </key>','').replace('</key>','</post>').replace('         </title>','</title>'))
finally:
    f.close()

In [32]:
# print(doc.toprettyxml(indent="  ").replace('<key name="','').replace('</link>\n','</link>\n      </rubric>\n').replace('\n      </key>','').replace('</key>','</post>').replace('         </title>','</title>'))

In [31]:
# print(doc.toprettyxml(indent="  ").replace('key name="post id=&quot;post_493852&quot;"','post id="post_493852"'))
# print(doc.toprettyxml(indent="  ").replace('<key name="','').replace('</link>\n','</link>\n      </rubric>\n').replace('\n      </key>','').replace('</key>','</post>').replace('         </title>','</title>'))